# Local Outlier Factor(LOF) and  Isolation Forest (IF)

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor


In [ ]:
df = pd.read_csv('a.csv')
df.shape
df.columns
df.describe
df.isnull().any()#.any()

In [ ]:
#check the sample distribution
sample = df.sample(frac=0.3,random_state=2000) #use 30% of dataset for checking
fraud = sample[sample['class'] == 1]
valid = sample[sample['class'] == 0]

outlier_fraction = (len(fraud)/float(len(valid)))
print('Outlier_fraction :{}%'.format(outlier_fraction*100)) #fraud over valid 
print('Valid transactions:{}'.format(len(sample[sample['class']==0])))

## data visualization

In [ ]:
df.hist(figsize=(15,15))
plt.show()

In [ ]:
corrmat = df.corr()
fig = plt.figure(figsize=(15,15))

sns.heatmap(corrmat,vamx=.6, square= True) #vmax is the max and min value you want to have for the scale 
plt.show()

In [ ]:
corrmat['class']

## feature selection

In [ ]:
cols = corrmat.keys()
cols_to_keep = []

for i in range(len(corrmat)):
    if abs(corrmat['class'][i])>0.01:
        cols_to_keep.append(cols[i])

len(cols_to_keep)
cols_to_keep

In [ ]:
# removing the 'Class' columnn from the features list, as it is the variable we wish to predict
cols = cols_to_keep[:-1]

In [ ]:
features = df[cols]
target = df['class']

## model training 

In [ ]:
#define random state
state = 2000

#define outlier detection tools to be compared
classfiers = {
    #isolate forest is unsuprvised learning method 
    'IF' : IsolationForest(max_samples = len(features),
                           contamination = outlier_fraction,
                           random_state = state),
    'LOF' : LocalOutlierFactor(
        n_neighbors = 20,
        contamination = outlier_fraction
    )
}

In [ ]:
#skipping the train, test split step since we wish the model to overfit on these features and learn
#a mathematical function to map the features 

n_outliers = len(fraud)

for i, (clf_name,clf) in enumerate(classfiers.items()):
    if clf_name == 'LOF':
        y_pred = clf.fit_predict(features)
        scores_pred = clf.negative_outlier_factor_

    else:
        clt.fit(features)
        scores_pred = clf.decision_function(features)
        y_pred = clf.predict(features)

    #reshape the predcition values to 0 for valid, 1 for fraud 
    y_pred[y_pred==1] ==0
    y_pred[y_pred==-1] == 1

    n_errors = (y_pred != target.sum()

    # Run classification metrics
    print('Classifier {0}: \nNumber of Errors: {1}'.format(clf_name, n_errors))
    print('Accuracy: {0}%\n'.format(accuracy_score(target, y_pred)*100))
    print(classification_report(target, y_pred))

classifiers.items()：这个方法返回一个包含字典中所有键值对的迭代器。每个元素都是一个包含键和值的元组（tuple），例如：('IF', IsolationForest(max_samples = len(features),
                           contamination = outlier_fraction,
                           random_state = state))。

enumerate 是一个内置函数，用于将一个可迭代的数据对象（如列表、元组、字符串等）组合为一个索引序列，同时列出数据和数据下标。在这里，enumerate 作用于 classifiers.items() 的结果上，这意味着它会生成一系列带有索引的元组，每个元组的第一个元素是索引（从0开始），第二个元素是来自 classifiers.items() 的键值对元组。

(i, (clf_name, clf))：这是一个解包表达式，用于在循环中提取索引和键值对。i 是索引，clf_name 是分类器的名称（字典的键），clf 是分类器实例本身（字典的值）。

# Gaussian distribution & Multivairate Gaussian distribution

In [22]:
from scipy.stats import multivariate_normal


In [ ]:
cols = ['1','b','c','d']
detection_data = df[cols]

#Standardization 
from sklearn.preprocessing import StandarScaler

scaler = StandarScaler()
detection_data_scaled = scaler.fit_transform(decetion_data)

#Mean and coveriance
mu =np.mean(detection_data_scaled,axis=0)
cov = np.cov(detection_data_scaled,rowvar = False)

#Multivirate Gaussian distribution model
model = multivariate_normal(mean = mu, cov = cov, allow_singular = True)

#Comput P-value
p_value = model.pdf(detection_data_scaled)
#treshold 
threshold = np.percentile(p_values,1) #set 1% as the treshold 
#mark outliers 
outliers = p_values< threshold 

#result 
detection_data['outlier'] = outliers 
detection_data[detection_data['outlier'] == True

In [ ]:
#details 
detection_data[detection_data['outlier']==True]


### PCA

In [ ]:
from sklearn.decomposition import PCA

anomalies = detection_data[detection_data['outlier'] == True]

scaler = StandarScaler()
scaled_data = scaler.fit_transform(anomalies[cols])

#PCA
pca = PCA()
pca.fit(scaled_data)

#Cumulative variance ratio
cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)

#visualization
plt.figure(figsize=(10,6))
plt.plot(range(1,len(cumulative_variance_ratio) +1),cumulative_variance_ratio,marker = 'o')
plt.title('Cumulative Explained Variance Ratio by Principal Components')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.grid(True)
plt.show()

In [ ]:
# Optimal number of components is 3
pca = PCA(n_components=3)
principal_components = pca.fit_transform(scaled_data)

### clustering 

In [ ]:
from sklearn.cluster import KMeans 

#ELBOW
sse = []
k_range = range(1,11)
for k in k_range:
    kmeans = KMeans(n_cluster = k, random_state = 2000)
    kmeans.fit(principal_components)
    sse.append(kmeans.inertia_)

#visualization
plt.figure(figsize=(10,6))
plt.plot(k_range,see,marker ='o')
plt.title('Elbow method for optimal K')
plt.xlabel('Number of Clusters')
plt.ylabel('Sum of Squared Errors(SSE)')
plt.grid(True)
plt.show()

In [ ]:
optimal_k = 3
kmeans = KMeans(n_clusters = optimal_k,random_state=2000)
anomlies['cluster'] = kmeans.fit_predict(principal_components)

#Visualization 
plt.figure(figsize = (10,6))
sns.scatterplot(data=pca_df,x = 'PC1', y = 'PC2', hue = 'cluster')
plt.title('Kmeans clustering of PCA transformed Anomalies')
plt.show()

In [ ]:
# Cheack points data in each clusters
for cluster in range(optimal_k):
    cluster_data = anomalies[anomalies['cluster'] == cluster]
    print(f"Cluster {cluster} data points:")
    print(cluster_data)
    print("\n")

In [ ]:
# Discribe of each cluster
for cluster in range(optimal_k):
    cluster_data = anomalies[anomalies['cluster'] == cluster]
    cluster_mean = cluster_data[columns].mean()
    cluster_std = cluster_data[columns].std()
    print(f"Cluster {cluster} mean values:")
    print(cluster_mean)
    print("\n")
    print(f"Cluster {cluster} standard deviation values:")
    print(cluster_std)
    print("\n")

In [ ]:
#compare with the normal data points 
normal_data = detection_data[detection_data['outlier'] ==False]

normal_mean = normal_data[columns].mean()
normal_std = normal_data[columns].std()

print(f"Normal mean values:")
print(normal_mean)
print("\n")
print(f"Normal standard deviation values:")
print(normal_std)
print("\n")

# Seasonal and Trend decomposition using Loess

In [ ]:
df = df[df['kw'].notna()]

In [ ]:
plt.figure(figsize=(14,7))
sns.barplot(x=df.index, y = df['kw'])
plt.title('Total Hourly Consumption by Hour')
plt.xlabel('Hour of day')
plt.ylabel('Total Hourly consumption(KW)')
plt.show()

In [ ]:
from statsmodels.tsa.seasonal import STL
from itertools import product

seasonal_options = [7,13,23] #smoothness of the seasonal component
period_options = [24] #24hours

#initialize best parameters and residual 
best_parameters = None
min_residual_var=  np.inf

#loop
for seasonal, period in product(seasonal_options, period_options):
    stl = STL(df['kw'],seasonal = seasonal, period = period)
    result = stl.fit()

    #vairance
    residual_var = result.resid.var()

    #update
    if residual_var < min_residual_var:
        min_residual_var= residual_var
        best_parameters = (seasonal,period)

best_parametsers, min_residual_var

In [ ]:
stl = STL(df['kW'], seasonal=7, period=24)  
result = stl.fit()

# Get trend, seasonal and residual
trend = result.trend
seasonal = result.seasonal
residual = result.resid

# Anomaly: Residual > 2 * std
residual_std = residual.std()
anomalies = residual[abs(residual) > 2 * residual_std]

In [ ]:
df['trend'] = trend 
df['seasonal'] = seasonal 
df['residual'] = residual 
df['is_anomaly'] = df['residual'].apply(lambda x: 'Yes' if abs(x)>2*residual_std else 'NO')

In [ ]:
# Figure
plt.figure(figsize=(14, 10))

# Initial data
plt.subplot(411)
plt.plot(df.index, df['kW'], label='Original', color='blue')
plt.title('Original Data')
plt.legend()

# Trend component
plt.subplot(412)
plt.plot(df.index, df['trend'], label='Trend', color='red')
plt.title('Trend Component')
plt.legend()

# Seasonal component
plt.subplot(413)
plt.plot(df.index, df['seasonal'], label='Seasonal', color='green')
plt.title('Seasonal Component')
plt.legend()

# Residual component
plt.subplot(414)
plt.plot(df.index, df['residual'], label='Residual', color='magenta')
plt.title('Residual Component')
plt.legend()

# Mark anomaly data
for time_point in anomalies.index:
    plt.axvline(x=time_point, color='gray', linestyle='--', linewidth=0.8)

plt.tight_layout()
plt.show()

In [ ]:
anomaly_data = df[df['is_anomaly'] == 'Yes']
anomaly_data

In [ ]:
# hourly
plt.figure(figsize=(18, 5))
plt.subplot(131)
sns.countplot(x=anomaly_data['hour'], data=anomaly_data)
plt.title('Hourly Distribution of Anomalies')
plt.xlabel('Hour of the Day')
plt.ylabel('Count of Anomalies')

# monthly
plt.subplot(132)
sns.countplot(x=anomaly_data['month'], data=anomaly_data)
plt.title('Monthly Distribution of Anomalies')
plt.xlabel('Month')
plt.ylabel('Count of Anomalies')

# weekday
plt.subplot(133)
sns.countplot(x=anomaly_data['weekday'], data=anomaly_data)
plt.title('Weekday Distribution of Anomalies')
plt.xlabel('Day of the Week')
plt.ylabel('Count of Anomalies')

plt.tight_layout()
plt.show()